# Legal Deidentification

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/10.Deidentification.ipynb)

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Saving latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json to latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json


In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up John Snow Labs home in /home/ckl/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-ocr/spark_ocr-4.0.1-py3-none-any.whl --force-reinstall"
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-nlp-internal/spark_nlp_internal-4.1.0-py3-none-any.whl --force-reinstall"
Installing spark-nlp to /usr/bin/python3
Installing pyspark to /usr/bin/python3
Installed 3 products:
🕶 Spark-OCR installed! ✅ Empower your NLP with a set of eyes 
💊 Spark-Healthcare installed! ✅ Heal the planet with NLP! 
🚀 Spark-NLP installed! ✅ State of the art NLP at scale 
🍥 If you are on Google Colab, please restart your Notebook for cha

In [ ]:
from johnsnowlabs import * 
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored new John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_2_for_Spark-Healthcare_Spark-OCR.json
👌 Launched SparkSession with Jars for: 🚀Spark-NLP, 💊Spark-Healthcare, 🕶Spark-OCR


In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to start the session with custom params as in start function above
def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()

#spark = start(SECRET)


# Deidentification Model

Some legal information can be considered sensitive. (e.g.,document, organization, address, signer)

In [6]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
    #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained("legner_contract_doc_parties", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"}) # "ALIAS" are secondary names of companies, so let's extract them also as PARTY

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]
legner_contract_doc_parties download started this may take some time.
[OK!]


### Pretrained NER models extracts:
- Document
- Date
- Party (Organization Name)
- Alias

In [7]:
legal_ner.getClasses()

['O',
 'I-DOC',
 'B-EFFDATE',
 'B-ALIAS',
 'I-ALIAS',
 'B-PARTY',
 'I-EFFDATE',
 'I-PARTY',
 'B-DOC']

In [8]:
text = """THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C., a limited liability company organized and existing under the laws of the State of Delaware """

In [9]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [10]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.ner.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

In [11]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+---------+-----+
|ner_label|count|
+---------+-----+
|O        |31   |
|I-PARTY  |5    |
|I-EFFDATE|3    |
|I-DOC    |2    |
|B-PARTY  |1    |
|B-DOC    |1    |
|B-EFFDATE|1    |
+---------+-----+



### Check extracted sensitive entities

In [12]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------------------+---------+
|chunk                                 |ner_label|
+--------------------------------------+---------+
|STRATEGIC ALLIANCE AGREEMENT          |DOC      |
|December 14, 2016                     |EFFDATE  |
|Hyatt Franchising Latin America, L.L.C|PARTY    |
+--------------------------------------+---------+



## Masking and Obfuscation

### Replace these enitites with Tags

In [13]:
ner_converter = legal.NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk") 

deidentification = legal.DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [14]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [15]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [16]:
result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i..."


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>

Let's try each of these and compare the results:

In [17]:
#deid model with "entity_labels"
deid_entity_labels= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model_deid = deidPipeline.fit(empty_data)

In [18]:
policy_result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [19]:
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|   deid_entity_label|                 aux|    deid_same_length|   deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 8, **...|[{document, 0, 24...|[{document, 0, 17...|
+--------------------+--------------------+--------------------+----

In [20]:
policy_result.select(F.explode(F.arrays_zip(policy_result.sentence.result, 
                                            policy_result.deid_entity_label.result, 
                                            policy_result.deid_same_length.result, 
                                            policy_result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i...","THIS [**************************] (""Agreement""...","THIS **** (""Agreement"") is made and entered in..."


### Mapping Column

In [21]:
result.select("aux").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aux                                                                                                                                                                                                                                                                                                                                   

In [22]:
result.select(F.explode(F.arrays_zip(result.aux.metadata, 
                                     result.aux.result, 
                                     result.aux.begin, 
                                     result.aux.end)).alias("cols")) \
      .select(F.expr("cols['0']['originalChunk']").alias("chunk"),
              F.expr("cols['0']['beginOriginalChunk']").alias("beginChunk"),
              F.expr("cols['0']['endOriginalChunk']").alias("endChunk"),
              F.expr("cols['1']").alias("label"),
              F.expr("cols['2']").alias("beginLabel"),
              F.expr("cols['3']").alias("endLabel")).show(truncate=False)

+--------------------------------------+----------+--------+---------+----------+--------+
|chunk                                 |beginChunk|endChunk|label    |beginLabel|endLabel|
+--------------------------------------+----------+--------+---------+----------+--------+
|STRATEGIC ALLIANCE AGREEMENT          |5         |32      |<DOC>    |5         |9       |
|December 14, 2016                     |79        |95      |<EFFDATE>|56        |64      |
|Hyatt Franchising Latin America, L.L.C|114       |151     |<PARTY>  |83        |89      |
+--------------------------------------+----------+--------+---------+----------+--------+



## Reidentification

We can use `ReIdentification` annotator to go back to the original sentence.

In [23]:
reIdentification = legal.ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [24]:
reid_result = reIdentification.transform(result)

In [25]:
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|        deidentified|                 aux|            original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|[{document, 0, 24...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----

In [26]:
print(text)

reid_result.select('original.result').show(truncate=False)

THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C., a limited liability company organized and existing under the laws of the State of Delaware 
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[T

## Using multiple NER in the same pipeline

In [27]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained("legner_contract_doc_parties", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"})

ner_signers = legal.NerModel.pretrained("legner_signers", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_signers") 
    #.setLabelCasing("upper")

ner_converter_signers = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = legal.ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

deidentification = legal.DeIdentification() \
    .setInputCols(["sentence", "token", "deid_merged_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("mask")\
    .setIgnoreRegex(True)


nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]
legner_contract_doc_parties download started this may take some time.
[OK!]
legner_signers download started this may take some time.
[OK!]


In [28]:
text = """ENTIRE AGREEMENT.  This Agreement contains the entire understanding of the parties hereto with respect to the transactions and matters contemplated hereby, supersedes all previous Agreements between i-Escrow and 2TheMart concerning the subject matter.

2THEMART.COM, INC.:                         I-ESCROW, INC.:

By:Dominic J. Magliarditi                By:Sanjay Bajaj Name: Dominic J. Magliarditi                Name: Sanjay Bajaj Title: President                            Title: VP Business Development Date: 6/21/99                               Date: 6/11/99 """ 

In [29]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# legal_ner
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------+---------+
|chunk           |ner_label|
+----------------+---------+
|ENTIRE AGREEMENT|DOC      |
+----------------+---------+



In [30]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# ner_signers
result.select(F.explode(F.arrays_zip(result.ner_signer_chunk.result, 
                                     result.ner_signer_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+--------------+
|chunk                  |ner_label     |
+-----------------------+--------------+
|INC                    |PARTY         |
|J. Magliarditi         |SIGNING_PERSON|
|Bajaj                  |SIGNING_PERSON|
|Dominic J. Magliarditi |SIGNING_PERSON|
|Sanjay Bajaj           |SIGNING_PERSON|
|President              |SIGNING_TITLE |
|VP Business Development|SIGNING_TITLE |
+-----------------------+--------------+



In [31]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# merged_chunk
result.select(F.explode(F.arrays_zip(result.deid_merged_chunk.result, 
                                     result.deid_merged_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+--------------+
|chunk                  |ner_label     |
+-----------------------+--------------+
|ENTIRE AGREEMENT       |DOC           |
|INC                    |PARTY         |
|J. Magliarditi         |SIGNING_PERSON|
|Bajaj                  |SIGNING_PERSON|
|Dominic J. Magliarditi |SIGNING_PERSON|
|Sanjay Bajaj           |SIGNING_PERSON|
|President              |SIGNING_TITLE |
|VP Business Development|SIGNING_TITLE |
+-----------------------+--------------+



In [32]:
result.select(F.explode(F.arrays_zip(result.sentence.result, 
                                     result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,ENTIRE AGREEMENT.,<DOC>.
1,This Agreement contains the entire understandi...,This Agreement contains the entire understandi...
2,"2THEMART.COM, INC.: I-...","2THEMART.COM, INC.: I-..."


## Obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensitive entities with random values of the same type. 


In [33]:
# This is the obfuscation dict for the new entities
obs_lines = """CTO#SIGNING_TITLE
Project Manager#SIGNING_TITLE
Sales Manager#SIGNING_TITLE
Business Manager#SIGNING_TITLE
Coordinator#SIGNING_TITLE
Officer#SIGNING_TITLE
Legal Agreement#DOC
Contract#DOC
Estate Document#DOC
official Document#DOC
Deed of Covenant#DOC
TURER INC#PARTY
Clark llc.#PARTY
SESA CO.#PARTY
John Snow Labs Inc#PARTY
MGT Trust Company, LLC.#PARTY
JAMES TURNER#SIGNING_PERSON
Juan Garcia#SIGNING_PERSON
Benjamin Dean#SIGNING_PERSON
Tommy Lee#SIGNING_PERSON
Dorothy Keen#SIGNING_PERSON
("AGREEMENT")#ALIAS
("TRADE COMPANY")#ALIAS
(the" Agreement")#ALIAS
("private company")#ALIAS
(the "Contract")#ALIAS
26-06-1990#EFFDATE
03/08/2025#EFFDATE
01/01/2045#EFFDATE
11/7/2016#EFFDATE
12-12-2022#EFFDATE """

with open ('obfuscate.txt', 'w') as f:
    f.write(obs_lines)

In [34]:
ner_converter_signers = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = legal.ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

obfuscation = legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_signer_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate.txt')\
    .setObfuscateRefSource("both") #default: "faker"


nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      obfuscation])

obfuscation_model = nlpPipeline.fit(empty_data)

In [35]:
text = """"Newegg" "Allied" Newegg Inc. Allied Esports International, Inc. By  Mitesh Patel By:  Judson Hannigan Name: Mitesh Patel Name: Judson Hannigan Title: VP, Marketing Title: CEO Newegg Inc. Allied Esports International, Inc. """

In [36]:
result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"""Newegg"" ""Allied"" Newegg Inc.","""Newegg"" ""Allied"" SESA CO.."
1,"Allied Esports International, Inc.",TURER INC.
2,By Mitesh Patel By: Judson Hannigan Name: Mi...,By Benjamin Dean By: JAMES TURNER Name: Benj...
3,"Allied Esports International, Inc.",TURER INC.


## Use full pipeline in the Light model

In [37]:
light_model = LightPipeline(model)
annotated_text = light_model.annotate(text)
annotated_text['deidentified']

['"<PARTY>" "<PARTY>" <PARTY>.',
 '<PARTY>.',
 'By  <SIGNING_PERSON> By:  <SIGNING_PERSON> Name: <SIGNING_PERSON> Name: <SIGNING_PERSON> Title: <SIGNING_TITLE> Title: <SIGNING_TITLE> <PARTY>.',
 '<PARTY>.']

In [38]:
obf_light_model = LightPipeline(obfuscation_model)
annotated_text = obf_light_model.annotate(text)
annotated_text['deidentified']

['"Newegg" "Allied" SESA CO..',
 'TURER INC.',
 'By  Benjamin Dean By:  JAMES TURNER Name: Benjamin Dean Name: JAMES TURNER Title: Sales Manager Title: CTO SESA CO..',
 'TURER INC.']

# Save the Pipeline and Use it from Your Local

In [39]:
model.write().overwrite().save('pipeline_deid')

In [40]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline.from_disk("pipeline_deid")

In [41]:
data = spark.createDataFrame([[text]]).toDF("text")

In [42]:
deid_pipeline.model.stages

[DocumentAssembler_245c3f7290b9,
 SentenceDetector_70fd2ed48a15,
 REGEX_TOKENIZER_620c689e0082,
 ROBERTA_EMBEDDINGS_b915dff90901,
 MedicalNerModel_93f728ff96e5,
 NER_CONVERTER_b4a7b0bab740,
 MedicalNerModel_2b2f0f671f99,
 NerConverter_8e11755735a1,
 MERGE_eebf13964608,
 DE-IDENTIFICATION_30bb0f5aa5e6]

In [43]:
deid_pipeline.model.transform(data).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|         ner_signers|    ner_signer_chunk|   deid_merged_chunk|        deidentified|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|"Newegg" "Allied"...|[{document, 0, 22...|[{document, 0, 28...|[{token, 0, 0, ",...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 1, 6, Ne...|[{named_entity, 0...|[{chunk, 18, 27, ...|[{chunk, 1, 6, Ne...|[{document, 0, 27...|
+--------------------+--------------------+--------------------+----

# Pretrained Deidentification Pipeline

We have this pipeline can be used to deidentify legal information from texts.The legal information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `DOC`, `EFFDATE`, `PARTY`, `ALIAS`, `SIGNING_PERSON`, `SIGNING_TITLE`, `COUNTRY`, `CITY`, `STATE`, `STREET`, `ZIP`, `EMAIL`, `FAX`, `LOCATION-OTHER`, `DATE`,`PHONE` entities.

In [44]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("legpipe_deid", "en", "legal/models")

legpipe_deid download started this may take some time.
Approx size to download 900.4 MB
[OK!]


Py4JJavaError: ignored

In [ ]:
deid_pipeline.model.stages

In [ ]:
text= """CARGILL, INCORPORATED

By:     Pirkko Suominen



Name: Pirkko Suominen Title: Director, Bio Technology Development  Center,  Date:   10/19/2011

BIOAMBER, SAS

By:     Jean-François Huc



Name: Jean-François Huc  Title: President Date:   October 15, 2011

email : jeanfran@gmail.com
phone : 18087339090 """

In [ ]:
deid_res= deid_pipeline.annotate(text)

In [ ]:
deid_res.keys()

In [ ]:
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"], 
                          deid_res["deidentified"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"], 
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df